In [1]:
!pip install -q efficientnet
!pip install iterative-stratification

In [2]:
%%time 
%load_ext autoreload
%autoreload 2
import math, re, os, gc, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 

import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow.keras.layers as L
import tensorflow.keras.applications as app 
import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

CPU times: user 3.57 s, sys: 827 ms, total: 4.39 s
Wall time: 10.6 s


In [3]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

GCS_DS_PATH = KaggleDatasets().get_gcs_path()

In [4]:
EPOCHS = 30
BATCH_SIZE = 16*strategy.num_replicas_in_sync

In [5]:
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

In [6]:
train = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
sample_submission = pd.read_csv('../input/plant-pathology-2020-fgvc7/sample_submission.csv')

In [7]:
train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values
train_labels = train.loc[:, "healthy":].values

In [8]:
def decode_image(filename, label=None, image_size=(300,300)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    if label is None:
        return image
    else:
        return image, label
    
def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.resize_with_crop_or_pad(image, 310, 310)
    image = tf.image.random_crop(image, size=[300, 300, 3])
    image = tf.image.random_brightness(image, max_delta=0.5)
    if label is None:
        return image
    else:
        return image, label

In [9]:
def dataset(train_paths, train_labels, valid_paths, valid_labels):
    train_dataset = (
                    tf.data.Dataset
                    .from_tensor_slices((train_paths, train_labels))
                    .map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                    .cache()
                    .map(data_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                    .repeat()
                    .shuffle(512)
                    .batch(BATCH_SIZE)
                    .prefetch(tf.data.experimental.AUTOTUNE)
                    )

    valid_dataset = (
                    tf.data.Dataset
                    .from_tensor_slices((valid_paths, valid_labels))
                    .map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                    .cache()
                    .batch(64)
                    .prefetch(tf.data.experimental.AUTOTUNE)
                    )
    return train_dataset, valid_dataset

In [10]:
test_dataset = (
                tf.data.Dataset
                .from_tensor_slices((test_paths))
                .map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                .cache()
                .batch(BATCH_SIZE)
                .prefetch(tf.data.experimental.AUTOTUNE)
                )

### Efficient Net B7 with Visual Attention

In [11]:
def create_model_efficientnet_attention(image_batch):
    with strategy.scope():
        # input layer
        in_lay = L.Input(image_batch.shape[1:])

        # pretrained model
        base_pretrained_model = efn.EfficientNetB7(input_shape=(720,720,3),weights='imagenet',include_top=False)
        base_pretrained_model.trainable = False
        pt_depth = base_pretrained_model.get_output_shape_at(0)[-1]
        pt_features = base_pretrained_model(in_lay)

        #batch normalization layer
        bn_features = L.BatchNormalization()(pt_features)

        #attention layer
        attn_layer = L.Conv2D(128, kernel_size = (1,1), padding = 'same', activation = 'relu')(bn_features)
        attn_layer = L.Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
        attn_layer = L.Conv2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid')(attn_layer)
        up_c2_w = np.ones((1, 1, 1, pt_depth))
        up_c2 = L.Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', activation = 'linear', use_bias = False, weights = [up_c2_w])
        up_c2.trainable = False
        attn_layer = up_c2(attn_layer)

        #global average pooling layers
        mask_features = L.multiply([attn_layer, bn_features])
        gap_features = L.GlobalAveragePooling2D()(mask_features)
        gap_mask = L.GlobalAveragePooling2D()(attn_layer)

        # to account for missing values from the attention model
        gap = L.Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
        gap_dr = L.Dropout(0.4)(gap)

        #final layers
        dr_steps = L.Dropout(0.4)(L.Dense(1024, activation = 'elu')(gap_dr))
        out_layer = L.Dense(train_labels.shape[1], activation = 'softmax')(dr_steps)
        model = tf.keras.models.Model(inputs = [in_lay], outputs = [out_layer])

        model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

### Efficient Net B7

In [12]:
def create_model():
    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(300, 300, 3),
                weights='imagenet',
                include_top=False
            ),
            L.GlobalAveragePooling2D(),
            L.Dense(train_labels.shape[1], activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss = 'categorical_crossentropy',
            metrics=['categorical_accuracy']
        )
    return model

In [13]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, 
               lr_min=0.000001, lr_rampup_epochs=20, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

In [14]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE

In [15]:
nfold = 5
all_predictions = []
mskf = MultilabelStratifiedKFold(n_splits=nfold, random_state=2020, shuffle=True)
for i, (train_idx, val_idx) in enumerate(mskf.split(train_paths, train_labels)):
    print(f"Fold - {i+1}/{nfold}")
    tr_paths = train_paths[train_idx]
    tr_labels = train_labels[train_idx]
    val_paths = train_paths[val_idx]
    val_labels = train_labels[val_idx]
    print("Creating Dataset ..")
    train_dataset, valid_dataset = dataset(tr_paths, tr_labels, val_paths, val_labels)
    print("Done.!")
    
    print("Training Mode ..")
    model = create_model()
    history = model.fit(train_dataset, 
                        epochs=EPOCHS, 
                        validation_data=valid_dataset, 
                        steps_per_epoch=STEPS_PER_EPOCH, 
                        callbacks=[lr_schedule])
    print("Evaluation Mode ..")
    print(model.evaluate(valid_dataset))
    
    # unfreeze layers and train again
    model.layers[1].trainable = True
    limit = 500
    for i in range(limit): model.layers[0].layers[i].trainable = False
    for i in range(limit, 806): model.layers[0].layers[i].trainable = True
    with strategy.scope():
        model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print("Finetuning Mode ..")
    fine_tune_epochs = 15
    total_epochs =  EPOCHS + fine_tune_epochs
    ft_history = model.fit(train_dataset,
                           validation_data=valid_dataset, 
                           epochs=total_epochs, 
                           steps_per_epoch=STEPS_PER_EPOCH, 
                           callbacks=[lr_schedule], 
                           initial_epoch =  history.epoch[-1])
    print("Evaluation Mode ..")
    print(model.evaluate(valid_dataset))
    all_predictions.append(model.predict(test_dataset))
    gc.collect()

Fold - 1/5
Creating Dataset ..
Done.!
Training Mode ..
258441216/258434480 [==============================] - 9s 0us/step
Train for 14 steps, validate for 6 steps

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/30
14/14 [==============================] - 323s 23s/step - loss: 1.3808 - categorical_accuracy: 0.2829 - val_loss: 1.3884 - val_categorical_accuracy: 0.2665

Epoch 00002: LearningRateScheduler reducing learning rate to 3.95e-05.
Epoch 2/30
14/14 [==============================] - 7s 479ms/step - loss: 1.2915 - categorical_accuracy: 0.4911 - val_loss: 1.3090 - val_categorical_accuracy: 0.4176

Epoch 00003: LearningRateScheduler reducing learning rate to 6.9e-05.
Epoch 3/30
14/14 [==============================] - 7s 477ms/step - loss: 1.0163 - categorical_accuracy: 0.7461 - val_loss: 0.8909 - val_categorical_accuracy: 0.7555

Epoch 00004: LearningRateScheduler reducing learning rate to 9.849999999999998e-05.
Epoch 4/30
14/14 [========================

In [16]:
preds = np.array(all_predictions).sum(axis=0)/5

In [17]:
sample_submission.loc[:, 'healthy':] = preds
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,8.391365e-07,2.081831e-04,9.997902e-01,7.596158e-07
1,Test_1,8.562303e-09,6.850918e-07,9.999992e-01,3.853503e-08
2,Test_2,7.287722e-09,1.296874e-07,4.565488e-10,9.999999e-01
3,Test_3,9.999942e-01,1.342393e-06,4.113271e-06,4.214021e-07
4,Test_4,1.122802e-07,1.135226e-04,9.998863e-01,8.218198e-09
